# 爽了，这里全靠定义函数，可以直接拿过来使用了
### DATASET can be downloaded here -> https://www.kaggle.com/vetrirah/customer


1. 了解问题陈述和导入包和数据集。

2. 执行 EDA（探索性数据分析） - 了解数据集：
    *  探索训练和测试数据并了解每列/特征表示的内容。
    *  检查数据集中目标列的不平衡。
    *  可视化计数图和唯一值以从数据集推断。

3. 如果存在，从训练数据中删除重复行。

4. 填充/估算缺失值连续 - 平均值/中值/任何特定值和分类 - 其他/前向填充/后向填充。

5. 特征工程
    * 特征选择 - 选择最重要的现有特征。
    * 特征创建 - 从现有特征创建新特征。
    * 使用预测器（独立）和目标（相关）将训练数据拆分为训练和验证数据。

6. 数据编码 - 标签编码、OneHot 编码和数据缩放 - MinMaxScaler、StandardScaler、RobustScaler

7. 为多类分类问题创建基线 ML 模型

8. 改进 ML 模型，使用 MODEL Evaluation METRIC - “Accuracy” 进行微调并预测 Traget “Outcome”


## 1.  Understand the Problem Statement & Import Packages and Datasets :

In [1]:
# Import Required Python Packages :

# Scientific and Data Manipulation Libraries :

import numpy as np
import pandas as pd

# Data Viz & Regular Expression Libraries :

import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

# Scikit-Learn ML Libraries :

from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.metrics import *

# Garbage Collection Libraries :

import gc

# Boosting Algorithm Libraries :

from xgboost                          import XGBClassifier
from catboost                         import CatBoostClassifier
from lightgbm                         import LGBMClassifier
from sklearn.ensemble                 import RandomForestClassifier, VotingClassifier
from sklearn.metrics                  import accuracy_score
from sklearn.model_selection          import StratifiedKFold,KFold

In [2]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/customer/Test.csv
/kaggle/input/customer/Train.csv
/kaggle/input/customer/sample_submission.csv


In [98]:
# Load data 

train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sub = pd.read_csv('sample_submission.csv')
print(train.columns)
print(train.shape)
print(test.shape)

Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')
(8068, 11)
(2627, 10)


## 2. Perform EDA (Exploratory Data Analysis) - Understanding the Datasets :    

### 2.1 Explore Train and Test Data and get to know what each Column / Feature denotes :

In [3]:
# Python Method 1 : Displays Data Information :

def display_data_information(data, data_types, dataframe_name):
    print(" Information of ",dataframe_name,": Rows = ",data.shape[0],"| Columns = ",data.shape[1],"\n")
    data.info()
    print("\n")
    for VARIABLE in data_types :
        data_type = data.select_dtypes(include=[ VARIABLE ]).dtypes
        if len(data_type) > 0 :
            print(str(len(data_type))+" "+VARIABLE+" Features\n"+str(data_type)+"\n"  )        

# Display Data Information of "train" :

data_types  = ["float32","float64","int32","int64","object","category","datetime64[ns]"]
display_data_information(train, data_types, "train")

 Information of  train : Rows =  8068 | Columns =  11 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


2 float64 Features
Work_Experience    float64
Family_Size        float64
dtype: object

2 int64 Features
ID     int64
Age    int64
dtype: object

7 object Features
Gender   

In [4]:
# Display Data Information of "test" :

display_data_information(test, data_types, "test")

 Information of  test : Rows =  2627 | Columns =  10 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2627 entries, 0 to 2626
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2627 non-null   int64  
 1   Gender           2627 non-null   object 
 2   Ever_Married     2577 non-null   object 
 3   Age              2627 non-null   int64  
 4   Graduated        2603 non-null   object 
 5   Profession       2589 non-null   object 
 6   Work_Experience  2358 non-null   float64
 7   Spending_Score   2627 non-null   object 
 8   Family_Size      2514 non-null   float64
 9   Var_1            2595 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 205.4+ KB


2 float64 Features
Work_Experience    float64
Family_Size        float64
dtype: object

2 int64 Features
ID     int64
Age    int64
dtype: object

6 object Features
Gender            object
Ever_Married      object
Gradua

In [5]:
# Python Method 2 : Displays Data Head (Top Rows) and Tail (Bottom Rows) of the Dataframe (Table) :

def display_head_tail(data, head_rows, tail_rows):
    display("Data Head & Tail :")
    display(data.head(head_rows).append(data.tail(tail_rows)))
#     return True

# Displays Data Head (Top Rows) and Tail (Bottom Rows) of the Dataframe (Table)
# Pass Dataframe as "train", No. of Rows in Head = 3 and No. of Rows in Tail = 2 :

display_head_tail(train, head_rows=3, tail_rows=2)

'Data Head & Tail :'

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B
8067,461879,Male,Yes,37,Yes,Executive,0.0,Average,3.0,Cat_4,B


In [6]:
display_head_tail(test, head_rows=3, tail_rows=2)

'Data Head & Tail :'

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,458989,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,Cat_6
1,458994,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,Cat_6
2,458996,Female,Yes,69,No,NaN,0.0,Low,1.0,Cat_6
2625,467961,Male,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4
2626,467968,Female,No,43,Yes,Healthcare,9.0,Low,3.0,Cat_7


In [102]:
# Python Method 3 : Displays Data Description using Statistics :

def display_data_description(data, numeric_data_types, categorical_data_types):
    
    print("Data Description :")
    display(data.describe( include = numeric_data_types))
    print("")
    display(data.describe( include = categorical_data_types))

# Display Data Description of "train" :

#display_data_description(train, data_types[0:4], data_types[4:7])
print(train.groupby(['Segmentation','Gender']).size())
print(train.groupby(['Segmentation','Ever_Married']).size())
print(train.groupby(['Segmentation','Graduated']).size())
print(train.groupby(['Segmentation','Profession']).size())
print(train.groupby(['Segmentation','Spending_Score']).size())

Segmentation  Gender
A             Female     909
              Male      1063
B             Female     861
              Male       997
C             Female     922
              Male      1048
D             Female     959
              Male      1309
dtype: int64
Segmentation  Ever_Married
A             No               801
              Yes             1137
B             No               479
              Yes             1348
C             No               398
              Yes             1549
D             No              1607
              Yes              609
dtype: int64
Segmentation  Graduated
A             No            729
              Yes          1219
B             No            508
              Yes          1332
C             No            348
              Yes          1607
D             No           1437
              Yes           810
dtype: int64
Segmentation  Profession   
A             Artist            558
              Doctor            199
              Enginee

In [8]:
# Display Data Description of "test" :

display_data_description(test, data_types[0:4], data_types[4:7])

Data Description :


,ID,Age,Work_Experience,Family_Size
count,2627.000000,2627.000000,2358.000000,2514.000000
mean,463433.918919,43.649791,2.552587,2.825378
std,2618.245698,16.967015,3.341094,1.551906
min,458989.000000,18.000000,0.000000,1.000000
25%,461162.500000,30.000000,0.000000,2.000000
50%,463379.000000,41.000000,1.000000,2.000000
75%,465696.000000,53.000000,4.000000,4.000000
max,467968.000000,89.000000,14.000000,9.000000


,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1
count,2627,2577,2603,2589,2627,2595
unique,2,2,2,9,3,7
top,Male,Yes,Yes,Artist,Low,Cat_6
freq,1424,1520,1602,802,1616,1672


In [9]:
# Checking Percentage(%) of Common ID's  between train and test data using Unique train values :

print(np.intersect1d(train['ID'], test['ID']).shape[0]/train['ID'].nunique())
common_ids = len(set(test['ID'].unique()).intersection(set(train['ID'].unique())))
print("Common IDs : ",common_ids)

# Data Leak as out of 2627 Rows , there are 2332 ID's in Common；所以说测试集直接从给定的训练集中划分

print("Unique IDs : ",test.shape[0] - common_ids)

0.2890431333663857
Common IDs :  2332
Unique IDs :  295


In [10]:
testx = pd.merge(test,train,how='left', on = 'ID')

## 3.  Remove Duplicate Rows from Train data if present :

In [11]:
# Python Method 4 : Removes Data Duplicates while Retaining the First one - Similar to SQL DISTINCT :

def remove_duplicate(data):
    
    print("BEFORE REMOVING DUPLICATES - No. of Rows = ",data.shape[0])
    data.drop_duplicates(keep="first", inplace=True) 
    print("AFTER REMOVING DUPLICATES  - No. of Rows = ",data.shape[0])
    
    return data

# Remove Duplicates from "train" data :

train = remove_duplicate(train)

# No Duplicates at all !!!

BEFORE REMOVING DUPLICATES - No. of Rows =  8068
AFTER REMOVING DUPLICATES  - No. of Rows =  8068


## 4.  Fill/Impute Missing Values Continuous - Mean/Median/Any Specific Value & Categorical - Others/ForwardFill/BackFill :

In [13]:
# Let LightGBM Classifier Handle the Issues :

In [14]:
# Python Method 6 : Displays Unique Values in Each Column of the Dataframe(Table) :

def display_unique(data):
    for column in data.columns :
        
        print("No of Unique Values in "+column+" Column are : "+str(data[column].nunique()))
        print("Actual Unique Values in "+column+" Column are : "+str(data[column].sort_values(ascending=True,na_position='last').unique() ))
        print("NULL Values :")
        print(data[ column ].isnull().sum())
        print("Value Counts :")
        print(data[column].value_counts())
        print("")
        
# Displays Unique Values in Each Column of "train" :
# Check "train" data for Values of each Column - Long Form :

display_unique(train)

# Display this info in a Table Format - Improvements coming In Part 2

No of Unique Values in ID Column are : 8068
Actual Unique Values in ID Column are : [458982 458983 458984 ... 467972 467973 467974]
NULL Values :
0
Value Counts :
460799    1
466217    1
460042    1
462091    1
464144    1
         ..
463511    1
467609    1
461468    1
459421    1
460801    1
Name: ID, Length: 8068, dtype: int64

No of Unique Values in Gender Column are : 2
Actual Unique Values in Gender Column are : ['Female' 'Male']
NULL Values :
0
Value Counts :
Male      4417
Female    3651
Name: Gender, dtype: int64

No of Unique Values in Ever_Married Column are : 2
Actual Unique Values in Ever_Married Column are : ['No' 'Yes' nan]
NULL Values :
140
Value Counts :
Yes    4643
No     3285
Name: Ever_Married, dtype: int64

No of Unique Values in Age Column are : 67
Actual Unique Values in Age Column are : [18 19 20 21 22 23 25 26 27 28 29 30 31 32 33 35 36 37 38 39 40 41 42 43
 45 46 47 48 49 50 51 52 53 55 56 57 58 59 60 61 62 63 65 66 67 68 69 70
 71 72 73 74 75 76 77 78 79 80 8

In [15]:
# Check "train" data for Values of each Column - Short Form :
# Use Whichever you feel good working with :

for i in train:
    print(f"column {i} unique values {train[i].unique()}")

column ID unique values [462809 462643 466315 ... 465406 467299 461879]
column Gender unique values ['Male' 'Female']
column Ever_Married unique values ['No' 'Yes' nan]
column Age unique values [22 38 67 40 56 32 33 61 55 26 19 70 58 41 31 79 49 18 36 35 45 42 83 27
 28 47 29 57 76 25 72 48 74 59 39 51 30 63 52 60 68 86 50 43 80 37 46 69
 78 71 82 23 20 85 21 53 62 75 65 89 66 73 77 87 84 81 88]
column Graduated unique values ['No' 'Yes' nan]
column Profession unique values ['Healthcare' 'Engineer' 'Lawyer' 'Entertainment' 'Artist' 'Executive'
 'Doctor' 'Homemaker' 'Marketing' nan]
column Work_Experience unique values [ 1. nan  0.  4.  9. 12.  3. 13.  5.  8. 14.  7.  2.  6. 10. 11.]
column Spending_Score unique values ['Low' 'Average' 'High']
column Family_Size unique values [ 4.  3.  1.  2.  6. nan  5.  8.  7.  9.]
column Var_1 unique values ['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' nan 'Cat_5']
column Segmentation unique values ['D' 'A' 'B' 'C']


### Multi - Class Classification Problem - Target has more than 2 Categories - 
### Target - Segmentation has 4 Values of Customers ['D' 'A' 'B' 'C']

## 5.  Feature Engineering

### 5.1 Feature Selection - Selection of Most Important Existing Features
### 5.2 Feature Creation  - Creation  of New Features from the Existing Features / Predictors :

In [16]:
# Concatenate train and test data into single DataFrame - df :

train['is_train'] = 1
test['is_train'] = 0
df = pd.concat([train,test])

In [17]:
# Convert 2 Categorical(String) Columns 'City_Type','Employer_Category' using Label Encode Technique :
# Docs : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# Label encode category values

from sklearn.preprocessing import LabelEncoder

for i in ['Gender' , 'Ever_Married' , 'Graduated','Profession','Spending_Score','Var_1'  ]:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i].astype('str'))

In [18]:
# Mapping Values to Label ENCODED Values :

df['Segmentation'] = df['Segmentation'].map({'A':0,'B':1,'C':2,'D':3})

In [19]:
# Get Back train data from df with a condition on column is_train == 1 :

train = df[df['is_train'] == 1]

## 6.  Split Train Data into Train and Validation Data with Predictors(Independent) & Target(Dependent) :

In [20]:
# split train into 5 folds and apply random forest and check accuracy of each fold

predictor_train = train.drop(['Segmentation','is_train','ID'],axis=1)
target_train    = train['Segmentation']

In [21]:
predictor_test = test.drop(['is_train','ID'],axis=1)


## 7.  Data Encoding - Label Encoding :

In [22]:
def data_encoding( encoding_strategy , encoding_data , encoding_columns ):
    
    if encoding_strategy == "LabelEncoding":
        Encoder = LabelEncoder()
        for column in encoding_columns :
            encoding_data[ column ] = Encoder.fit_transform(tuple(encoding_data[ column ]))
        
    elif encoding_strategy == "OneHotEncoding":
#         display(encoding_data[encoding_columns])
        encoding_data = pd.get_dummies( encoding_data  )
        
    elif encoding_strategy == "TargetEncoding":
        ## Code Coming soon
        print("TargetEncoding")

    else :
        encoding_data = pd.get_dummies( encoding_data[encoding_columns]  )
        
    dtypes_list =['float64','float32','int64','int32']
    # BEST CODE : 0.6872386379302422
#     encoding_data.astype( dtypes_list[0] ).dtypes # UNCOMMENTED EARLIER
    # NEW CODE : 0.6872386379302422 - NO CHANGE !!!
    # encoding_data.astype( dtypes_list[0] ).dtypes - COMMENTED NOW
    
    return encoding_data

encoding_columns  = [ "Gender", "Ever_Married" , "Graduated", "Profession" , "Spending_Score", "Var_1" ]
encoding_strategy = [ "OneHotEncoding", "LabelEncoding", "TargetEncoding", "ELSE"]

predictor_train_encode = data_encoding( encoding_strategy[1] , predictor_train , encoding_columns )
predictor_test_encode  = data_encoding( encoding_strategy[1] , predictor_test ,  encoding_columns )

In [23]:
print("predictor_train_encode SHAPE   : ",predictor_train_encode.shape)
display("predictor_train_encode COLUMNS : ",predictor_train_encode.head())

print("predictor_test_encode SHAPE   : ",predictor_test_encode.shape)
display("predictor_test_encode COLUMNS : ",predictor_test_encode.head())

predictor_train_encode SHAPE   :  (8068, 9)


'predictor_train_encode COLUMNS : '

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,1,0,22,0,5,1.0,2,4.0,3
1,0,1,38,1,2,NaN,0,3.0,3
2,0,1,67,1,2,1.0,2,1.0,5
3,1,1,67,1,7,0.0,1,2.0,5
4,0,1,40,1,3,NaN,1,6.0,5


predictor_test_encode SHAPE   :  (2627, 9)


'predictor_test_encode COLUMNS : '

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,0,1,36,1,2,0.0,2,1.0,5
1,1,1,37,1,5,8.0,0,4.0,5
2,0,1,69,0,9,0.0,2,1.0,5
3,1,1,59,0,4,11.0,1,2.0,5
4,0,0,19,0,8,NaN,2,4.0,5


## 8.  Create Baseline ML Model :

In [25]:
# Mention Categorical Values of the Light GBM Model to Handle :
categorical_features = ["Gender", "Ever_Married" ,"Graduated" ,"Profession" ,"Spending_Score" ,"Var_1" ]

lgb_model = LGBMClassifier()

# Apply Stratified K-Fold Cross Validation where K=5 or n_splits=5 :
kf = StratifiedKFold(n_splits=10,shuffle=True,random_state=10)
acc = []

# Pass predictor_train,target_train for Cross Validation :
for fold,(t_id,v_id) in enumerate(kf.split(predictor_train,target_train)):
    
    # Split train and validation data :
    tx = predictor_train.iloc[t_id]; ty = target_train.iloc[t_id]
    vx = predictor_train.iloc[v_id]; vy = target_train.iloc[v_id]
    
    # Train/Fit the Data to LighGBM Model :
    lgb_model.fit(tx,ty, categorical_feature = categorical_features )
    
    # Predict the Validation Data to Train LighGBM Model :
    val_y = lgb_model.predict(vx)
    
    # Get Accuracy Score on Validation Data for Each Fold :
    acc_score = accuracy_score(vy,val_y)
    acc.append(acc_score)
    print(f"fold {fold} accuracy {acc_score}")

# Get Mean of Accuracy Score on Validation Data for All 5 Folds :
print(f"Mean accuracy score {np.mean(acc)}")

/Users/MC/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


fold 0 accuracy 0.5377942998760843
fold 1 accuracy 0.5154894671623296
fold 2 accuracy 0.5192069392812887
fold 3 accuracy 0.5514250309789344
fold 4 accuracy 0.5328376703841388
fold 5 accuracy 0.5130111524163569
fold 6 accuracy 0.5006195786864932
fold 7 accuracy 0.5291201982651796
fold 8 accuracy 0.5459057071960298
fold 9 accuracy 0.56575682382134
Mean accuracy score 0.5311166868068176


In [85]:
lgb_model = LGBMClassifier(
                                   boosting_type='gbdt', 
                                   max_depth=15, 
                                   learning_rate=0.15, 
                                   objective='multiclass', # Multi Class Classification
                                   random_state=100,  
                                   n_estimators=1000 ,
                                   reg_alpha=0, 
                                   reg_lambda=1, 
                                   n_jobs=-1
                                 )

In [66]:
 #调参
#parameters={'max_depth': range(5,16,2),'num_leaves':range(33,pow(2,10)+1,100)}

lgb_model = LGBMClassifier(
                                   boosting_type='gbdt', 
                                   max_depth=15, 
                                   learning_rate=0.1, 
                                   objective='multiclass', # Multi Class Classification
                                   random_state=100,  
                                   n_estimators=1000 ,
                                   reg_alpha=0, 
                                   reg_lambda=1, 
                                   n_jobs=-1
)
'''                               )
gsearch = GridSearchCV(lgb_model, param_grid=parameters, scoring='roc_auc', cv=5)
gsearch.fit(tx, ty)
print('参数的最佳取值:{0}'.format(gsearch.best_params_))
print('最佳模型得分:{0}'.format(gsearch.best_score_))
print(gsearch.cv_results_['mean_test_score'])
print(gsearch.cv_results_['params'])

'''  

"                               )\ngsearch = GridSearchCV(lgb_model, param_grid=parameters, scoring='roc_auc', cv=5)\ngsearch.fit(tx, ty)\nprint('参数的最佳取值:{0}'.format(gsearch.best_params_))\nprint('最佳模型得分:{0}'.format(gsearch.best_score_))\nprint(gsearch.cv_results_['mean_test_score'])\nprint(gsearch.cv_results_['params'])\n\n"

## 9. Improve ML Model,Fine Tune with MODEL Evaluation METRIC - "Accuracy" and Predict Target "Segmentation" :

In [73]:
# Mention Categorical Values of the Light GBM Model to Handle :
categorical_features = ["Gender", "Ever_Married" ,"Graduated" ,"Profession" ,"Spending_Score" ,"Var_1" ]

# Apply Stratified K-Fold Cross Validation where K=5 or n_splits=5 :
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=10)
acc = []

# Pass predictor_train,target_train for Cross Validation :
for fold,(t_id,v_id) in enumerate(kf.split(predictor_train,target_train)):
    
    # Split train and validation data :
    tx = predictor_train.iloc[t_id]; ty = target_train.iloc[t_id]
    vx = predictor_train.iloc[v_id]; vy = target_train.iloc[v_id]
    
    # Train/Fit the Data to LighGBM Model :
    lgb_model.fit(tx,ty, categorical_feature = categorical_features )
    
    # Predict the Validation Data to Train LighGBM Model :
    val_y = lgb_model.predict(vx)
    
    # Get Accuracy Score on Validation Data for Each Fold :
    acc_score = accuracy_score(vy,val_y)
    acc.append(acc_score)
    print(f"fold {fold} accuracy {acc_score}")

# Get Mean of Accuracy Score on Validation Data for All 5 Folds :
print(f"Mean accuracy score {np.mean(acc)}")

/Users/MC/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
fold 0 accuracy 0.5396530359355638
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
fold 1 accuracy 0.5433705080545229
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
fold 2 accuracy 0.5278810408921933
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
fold 3 accuracy 0.527588344699318
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
fold 4 accuracy 0.555486670799752
Mean accuracy score 0.53879592007627


In [86]:
def model_train_predict_submit( Classifiers_model_name, model_name ,X_train, y_train, X_test, target):
    
    categorical_features = ["Gender", "Ever_Married" ,"Graduated" ,"Profession" ,"Spending_Score" ,"Var_1"  ]
    Classifiers_model_name.fit( X_train, y_train , categorical_feature = categorical_features )
    final_predictions = Classifiers_model_name.predict( X_test )
    print(final_predictions)  
   
    Result_Promoted = pd.DataFrame({'ID': sub['ID'], target : final_predictions})
    Result_Promoted[ target ]=Result_Promoted[ target ].map({0: "A", 1: "B", 2: "C", 3: "D" })
    print(Result_Promoted[ target ].unique())
    Result_Promoted.to_csv(model_name +"_Labelling=Yes_Scaling=Yes"+".csv",index=False)
    return Result_Promoted

model_name       = "LGBM_Tuned_BEST"
model_classifier = lgb_model
sub = model_train_predict_submit( model_classifier, model_name, predictor_train_encode,target_train, predictor_test_encode, target = 'Segmentation')

/Users/MC/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[1. 2. 0. ... 0. 2. 3.]
['B' 'C' 'A' 'D']


## 10. Result Submission, Check Leaderboard & Improve "ACCURACY" :

In [87]:
sub1 = pd.merge(sub,testx,how='left',on='ID')
sub1.head()

,ID,Segmentation_x,Gender_x,Ever_Married_x,Age_x,Graduated_x,Profession_x,Work_Experience_x,Spending_Score_x,Family_Size_x,...,Gender_y,Ever_Married_y,Age_y,Graduated_y,Profession_y,Work_Experience_y,Spending_Score_y,Family_Size_y,Var_1_y,Segmentation_y
0,458989,B,Female,Yes,36,Yes,Engineer,0.0,Low,1.0,...,Female,Yes,42.0,Yes,Engineer,1.0,Low,1.0,Cat_6,B
1,458994,C,Male,Yes,37,Yes,Healthcare,8.0,Average,4.0,...,Male,Yes,38.0,Yes,Healthcare,8.0,Average,4.0,Cat_6,C
2,458996,A,Female,Yes,69,No,NaN,0.0,Low,1.0,...,Female,Yes,71.0,No,NaN,1.0,Low,1.0,Cat_6,A
3,459000,C,Male,Yes,59,No,Executive,11.0,High,2.0,...,Male,Yes,58.0,No,Executive,12.0,High,2.0,Cat_6,C
4,459001,D,Female,No,19,No,Marketing,NaN,Low,4.0,...,Female,No,20.0,No,Marketing,NaN,Low,4.0,Cat_6,C


In [88]:
sub['segmentation2'] = sub1['Segmentation_y']
sub.head()
n=0
for i in range(len(sub)):
    if sub.iloc[i,1]==sub.iloc[i,2]:
        n+=1
print(n/len(sub))

0.5824133993148077


In [89]:
sub['segmentation2'] = sub['segmentation2'].fillna('x')
for i in range(len(sub)):
    if sub.iloc[i,2] != 'x':
        sub.iloc[i,1] = sub.iloc[i,2]
        
sub[['ID','Segmentation']].to_csv('FINAL_LGBM_BEST_SUBMISSION_TUNED.csv',index = False)

In [96]:
from xgboost                          import XGBClassifier
from catboost                         import CatBoostClassifier
from lightgbm                         import LGBMClassifier
from sklearn.ensemble                 import RandomForestClassifier, VotingClassifier

class Customer:
    def init(self, gender,marriage,age,graduation,profession,work,spending,family,var_1,segmentation):
        self.Gender=gender
        self.Ever_Married=marriage
        self.Age=age
        self.Graduated=graduation
        self.Profession=profession
        self.Work_Experience=work
        self.Spending_Score=spending
        self.Family_Size=family
        self.Var_1=var_1
        self.Segmentation=segmentation
        if not isinstance(profession, str): raise TypeError('需要输入字符型')
        if not isinstance(spending, str): raise TypeError('请在low,average,high中选择')
        if not isinstance(age,int): raise TypeError('年龄为整数')
        if not isinstance(work,int): raise TypeError('工作经验为整数')
        if not isinstance(family,int): raise TypeError('工作经验为整数')
        if marriage not in ['Yes','No']:raise ValueError('请输入Yes or No')
        if graduation not in ['Yes','No']:raise ValueError('请输入Yes or No')
        if age < 15 or score > 100: raise ValueError('are you kidding')
    def printinfo(self):
        print("Hello, world!\n Gender: {}.\n Ever_Married: {}.\n Age: {}.\n Graduated:{}.\n Profession:{}.\n Work_Experience:{}.\n Spending_Score:{}.\n Family_Size:{}.\n Var_1:{}.\n Segmentation:{}".format(self.Gender,self.Ever_Married,self.Age,self.Graduated,self.Profession,self.Work_Experience,self.Spending_Score,self.Family_Size,self.Var_1,self.Segmentation) )
    def rf_pre(self,xtrain,ytrain):
        customer_list=[]
        ll=[self.Gender,self.Ever_Married,self.Age,self.Graduated,self.Profession,self.Work_Experience,self.Spending_Score,self.Family_Size,self.Var_1,self.Segmentation]
        for i in ll:
            if i in ['No','Male','Artist','Cat_1','Low']:customer_list.append(0)
            elif i in ['Yes','Female','Healthcare','Cat_2','Average']:customer_list.append(1)
            elif i in ['Cat_3','High','Entertainment']:customer_list.append(2)
            elif i in ['Cat_4','Engineer']:customer_list.append(3)
            elif i in ['Cat_5','Doctor']:customer_list.append(4)
            elif i in ['Cat_6','Lawyer']:customer_list.append(5)
            elif i =='Marketing':customer_list.append(7)
            elif i =='Homemaker':customer_list.append(8)
            else:customer_list.append(i)
        model=RandomForestClassifier()
        model.fit(xtrain,ytrain)
        pred=model.predict(ll)
        print('真实值:{}  预测值:{}'.format(self.Segmentation,pred))
        return(pred==self.Segmentation)
        
    def xgb_pre(self,xtrain,ytrain):
        customer_list=[]
        ll=[self.Gender,self.Ever_Married,self.Age,self.Graduated,self.Profession,self.Work_Experience,self.Spending_Score,self.Family_Size,self.Var_1,self.Segmentation]
        for i in ll:
            if i in ['No','Male','Artist','Cat_1','Low']:customer_list.append(0)
            elif i in ['Yes','Female','Healthcare','Cat_2','Average']:customer_list.append(1)
            elif i in ['Cat_3','High','Entertainment']:customer_list.append(2)
            elif i in ['Cat_4','Engineer']:customer_list.append(3)
            elif i in ['Cat_5','Doctor']:customer_list.append(4)
            elif i in ['Cat_6','Lawyer']:customer_list.append(5)
            elif i =='Marketing':customer_list.append(7)
            elif i =='Homemaker':customer_list.append(8)
            else:customer_list.append(i)
        model=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=5,
              learning_rate=0.3, max_delta_step=0, max_depth=4,
              min_child_weight=5, missing=1, n_estimators=500, n_jobs=4,
              nthread=None, objective='multi:softprob', eval_metric='mlogloss', use_label_encoder=False,
              random_state=0,reg_alpha=0, reg_lambda=1, scale_pos_weight=None, seed=None,
              silent=None, subsample=0.8, verbosity=None)
        model.fit(xtrain,ytrain)
        pred=model.predict(ll)
        print('真实值:{}  预测值:{}'.format(self.Segmentation,pred))
        return(pred==self.Segmentation)

    def lgb_pre(self,xtrain,ytrain):
        customer_list=[]
        ll=[self.Gender,self.Ever_Married,self.Age,self.Graduated,self.Profession,self.Work_Experience,self.Spending_Score,self.Family_Size,self.Var_1,self.Segmentation]
        for i in ll:
            if i in ['No','Male','Artist','Cat_1','Low']:customer_list.append(0)
            elif i in ['Yes','Female','Healthcare','Cat_2','Average']:customer_list.append(1)
            elif i in ['Cat_3','High','Entertainment']:customer_list.append(2)
            elif i in ['Cat_4','Engineer']:customer_list.append(3)
            elif i in ['Cat_5','Doctor']:customer_list.append(4)
            elif i in ['Cat_6','Lawyer']:customer_list.append(5)
            elif i =='Marketing':customer_list.append(7)
            elif i =='Homemaker':customer_list.append(8)
            else:customer_list.append(i)
        model= LGBMClassifier(
                                   boosting_type='gbdt', 
                                   max_depth=15, 
                                   learning_rate=0.1, 
                                   objective='multiclass', # Multi Class Classification
                                   random_state=100,  
                                   n_estimators=1000 ,
                                   reg_alpha=0, 
                                   reg_lambda=1, 
                                   n_jobs=-1)
        model.fit(xtrain,ytrain)
        pred=model.predict(ll)
        print('真实值:{}  预测值:{}'.format(self.Segmentation,pred))
        return(pred==self.Segmentation)

In [92]:
s=input('请输入你的名字：')
print(s=='崔')


请输入你的名字：崔博涵
False
